# Tutorial 4: Algoritmo Mapper

## Visualización Topológica de Datos Neuronales de Alta Dimensión

**Autor:** MARK-126  
**Nivel:** Avanzado  
**Tiempo estimado:** 120-150 minutos

---

## Objetivos de Aprendizaje

1. ✅ Comprender el algoritmo Mapper
2. ✅ Implementar Mapper desde cero
3. ✅ Aplicar a espacios de representación neural
4. ✅ Visualizar trayectorias cerebrales
5. ✅ Interpretar grafos de Mapper neurobiológicamente

---

## 1. ¿Qué es el Algoritmo Mapper?

### 1.1 Intuición

**Mapper** es un método de TDA para visualizar datos de alta dimensión como un **grafo simplificado** que captura la estructura topológica.

**Analogía:** Imagina tomar fotografías de un objeto 3D desde diferentes ángulos y luego reconstruir su forma.

### 1.2 ¿Por qué Mapper en Neurociencias?

En neurociencias tenemos:
- **Espacios de alta dimensión:** Actividad de miles de neuronas
- **Trayectorias complejas:** Estados cerebrales que evolucionan
- **Necesidad de visualización:** Entender estructura sin perder información

**Mapper nos permite:**
- Visualizar espacios neuronales de 1000+ dimensiones
- Identificar estados cerebrales como nodos
- Detectar transiciones como aristas
- Descubrir bucles (atractores, ciclos cognitivos)

---

## 2. Cómo Funciona Mapper

### Pasos del Algoritmo:

1. **Función de filtro:** Proyecta datos de alta dimensión a 1D o 2D
   - Ejemplos: PCA, densidad, distancia a punto
   
2. **Cover (cubrimiento):** Divide el rango del filtro en intervalos solapados
   - Parámetros: número de intervalos, % de solapamiento
   
3. **Clustering:** Agrupa puntos en cada intervalo
   - Algoritmo: DBSCAN, K-means, single-linkage
   
4. **Nerve:** Construye grafo donde:
   - **Nodos** = clusters
   - **Aristas** = clusters que comparten puntos

### Diagrama Conceptual:

```
Datos (alta dim) → Filtro → Cover → Clustering → Grafo de Mapper
     [X ∈ ℝⁿ]      f: ℝⁿ→ℝ   [{Uᵢ}]    [{Cⱼ}]        G=(V,E)
```

---

In [ ]:
# Importaciones
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# TDA y Mapper
import kmapper as km
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap

# Análisis
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.datasets import make_swiss_roll, make_s_curve

# Visualización
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(42)
print("✅ Bibliotecas importadas")
print(f"📦 KeplerMapper version: {km.__version__}")

---

## 3. Implementación Simple de Mapper

In [ ]:
def simple_mapper(data, filter_func, n_intervals=10, overlap=0.3, 
                 clustering=None):
    """
    Implementación didáctica del algoritmo Mapper.
    
    Parameters:
    -----------
    data : np.ndarray
        Datos de entrada (n_samples x n_features)
    filter_func : callable
        Función de filtro que mapea datos a 1D
    n_intervals : int
        Número de intervalos en el cover
    overlap : float
        Porcentaje de solapamiento entre intervalos (0-1)
    clustering : object
        Algoritmo de clustering (debe tener fit_predict)
    """
    if clustering is None:
        clustering = DBSCAN(eps=0.5, min_samples=3)
    
    # 1. Aplicar función de filtro
    filter_values = filter_func(data)
    
    # 2. Crear cover
    f_min, f_max = filter_values.min(), filter_values.max()
    interval_length = (f_max - f_min) / (n_intervals * (1 - overlap))
    step = interval_length * (1 - overlap)
    
    intervals = []
    for i in range(n_intervals):
        start = f_min + i * step
        end = start + interval_length
        intervals.append((start, end))
    
    # 3. Clustering en cada intervalo
    nodes = {}  # {node_id: [indices de puntos]}
    node_id = 0
    
    for interval_idx, (start, end) in enumerate(intervals):
        # Puntos en este intervalo
        mask = (filter_values >= start) & (filter_values <= end)
        indices = np.where(mask)[0]
        
        if len(indices) == 0:
            continue
        
        # Clustering
        subset = data[indices]
        labels = clustering.fit_predict(subset)
        
        # Crear nodos
        for label in set(labels):
            if label == -1:  # Ruido en DBSCAN
                continue
            cluster_mask = labels == label
            cluster_indices = indices[cluster_mask]
            nodes[node_id] = cluster_indices
            node_id += 1
    
    # 4. Construir grafo (nerve)
    G = nx.Graph()
    
    # Agregar nodos
    for nid in nodes.keys():
        G.add_node(nid, size=len(nodes[nid]))
    
    # Agregar aristas (si comparten puntos)
    node_ids = list(nodes.keys())
    for i, nid1 in enumerate(node_ids):
        for nid2 in node_ids[i+1:]:
            intersection = set(nodes[nid1]) & set(nodes[nid2])
            if len(intersection) > 0:
                G.add_edge(nid1, nid2, weight=len(intersection))
    
    return G, nodes, filter_values


# Función de filtro simple: primera componente de PCA
def pca_filter(data):
    pca = PCA(n_components=1)
    return pca.fit_transform(data).ravel()

print("✅ Funciones de Mapper definidas")

---

## 4. Ejemplo: Swiss Roll (Manifold Neuronal)

Vamos a usar el "Swiss Roll" como ejemplo de un espacio de representación neural.

---

In [ ]:
# Generar Swiss Roll
n_samples = 1000
swiss_roll, colors = make_swiss_roll(n_samples=n_samples, noise=0.1, random_state=42)

print(f"🎲 Generado Swiss Roll: {swiss_roll.shape}")
print(f"   Representa: Espacio de representación neural de 3D")

# Visualizar en 3D
fig = plt.figure(figsize=(14, 6))

# Plot 3D
ax1 = fig.add_subplot(121, projection='3d')
scatter = ax1.scatter(swiss_roll[:, 0], swiss_roll[:, 1], swiss_roll[:, 2],
                     c=colors, cmap='viridis', s=20, alpha=0.6)
ax1.set_title('Swiss Roll: Manifold Neural 3D', fontsize=12, fontweight='bold')
ax1.set_xlabel('Dimensión 1')
ax1.set_ylabel('Dimensión 2')
ax1.set_zlabel('Dimensión 3')
plt.colorbar(scatter, ax=ax1, label='Color intrínseco')

# Aplicar Mapper
print("\n⏳ Aplicando algoritmo Mapper...")
mapper_graph, mapper_nodes, filter_vals = simple_mapper(
    swiss_roll, 
    pca_filter,
    n_intervals=15,
    overlap=0.4,
    clustering=DBSCAN(eps=0.8, min_samples=5)
)

print(f"✅ Grafo de Mapper construido")
print(f"   Nodos: {mapper_graph.number_of_nodes()}")
print(f"   Aristas: {mapper_graph.number_of_edges()}")

# Visualizar grafo de Mapper
ax2 = fig.add_subplot(122)
pos = nx.spring_layout(mapper_graph, seed=42, k=1.5)
node_sizes = [mapper_graph.nodes[n]['size'] * 5 for n in mapper_graph.nodes()]

# Colorear nodos por valor medio del filtro
node_colors = []
for nid in mapper_graph.nodes():
    indices = mapper_nodes[nid]
    mean_filter = np.mean(filter_vals[indices])
    node_colors.append(mean_filter)

nx.draw_networkx_nodes(mapper_graph, pos, node_size=node_sizes,
                      node_color=node_colors, cmap='viridis',
                      alpha=0.8, ax=ax2)
nx.draw_networkx_edges(mapper_graph, pos, alpha=0.4, ax=ax2)

ax2.set_title('Grafo de Mapper', fontsize=12, fontweight='bold')
ax2.axis('off')

plt.tight_layout()
plt.show()

print("\n💡 El grafo de Mapper captura la estructura 'enrollada' del manifold!")

---

## 5. Aplicación: Trayectorias de Estados Cerebrales

In [ ]:
def generate_brain_trajectory(n_timepoints=500, n_neurons=50, 
                             trajectory_type='cyclic'):
    """
    Genera una trayectoria de estados cerebrales en espacio de alta dimensión.
    """
    if trajectory_type == 'cyclic':
        # Ciclo: descanso → atención → memoria → descanso
        t = np.linspace(0, 4*np.pi, n_timepoints)
        
        # Manifold base (círculo en 2D, embebido en n_neurons dimensiones)
        trajectory = np.zeros((n_timepoints, n_neurons))
        trajectory[:, 0] = 3 * np.cos(t)  # Primera dimensión
        trajectory[:, 1] = 3 * np.sin(t)  # Segunda dimensión
        
        # Agregar estructura en dimensiones adicionales
        for i in range(2, min(10, n_neurons)):
            trajectory[:, i] = 0.5 * np.sin(t * (i-1) / 2) * np.cos(t * i / 3)
        
        # Ruido en dimensiones restantes
        trajectory[:, 10:] = np.random.randn(n_timepoints, max(0, n_neurons-10)) * 0.3
        
        # Etiquetas de fase
        phase = (t % (2*np.pi)) / (2*np.pi)
        labels = np.zeros(n_timepoints, dtype=int)
        labels[phase < 0.25] = 0  # Descanso
        labels[(phase >= 0.25) & (phase < 0.5)] = 1  # Atención
        labels[(phase >= 0.5) & (phase < 0.75)] = 2  # Memoria
        labels[phase >= 0.75] = 0  # De vuelta a descanso
        
    elif trajectory_type == 'branching':
        # Bifurcación: estado inicial → decisión A o B
        trajectory = np.zeros((n_timepoints, n_neurons))
        labels = np.zeros(n_timepoints, dtype=int)
        
        # Fase 1: Común (0-200)
        t1 = np.linspace(0, 2, 200)
        trajectory[:200, 0] = t1
        labels[:200] = 0
        
        # Fase 2: Bifurcación
        mid = n_timepoints // 2
        
        # Rama A (200-350)
        t2 = np.linspace(0, np.pi, 150)
        trajectory[200:350, 0] = 2 + np.cos(t2)
        trajectory[200:350, 1] = np.sin(t2)
        labels[200:350] = 1
        
        # Rama B (350-500)
        t3 = np.linspace(0, np.pi, 150)
        trajectory[350:, 0] = 2 + np.cos(t3)
        trajectory[350:, 1] = -np.sin(t3)
        labels[350:] = 2
        
        # Ruido
        trajectory += np.random.randn(n_timepoints, n_neurons) * 0.2
    
    return trajectory, labels


# Generar trayectoria cíclica
print("🧠 Generando trayectoria de estados cerebrales cíclicos...\n")
brain_traj, phase_labels = generate_brain_trajectory(
    n_timepoints=400, 
    n_neurons=30,
    trajectory_type='cyclic'
)

print(f"✅ Trayectoria generada: {brain_traj.shape}")
print(f"   {brain_traj.shape[0]} puntos temporales")
print(f"   {brain_traj.shape[1]} neuronas (dimensiones)")
print(f"\n   Estados: 0=Descanso, 1=Atención, 2=Memoria")

### Aplicar Mapper a la Trayectoria

In [ ]:
# Usar KeplerMapper (biblioteca profesional)
mapper = km.KeplerMapper(verbose=0)

# Proyectar a 2D con PCA
pca = PCA(n_components=2)
brain_proj = pca.fit_transform(brain_traj)

print(f"📊 Varianza explicada por PCA: {pca.explained_variance_ratio_.sum():.1%}")

# Crear grafo de Mapper
print("\n⏳ Construyendo grafo de Mapper...")
mapper_graph_km = mapper.map(
    brain_proj,
    brain_traj,
    cover=km.Cover(n_cubes=15, perc_overlap=0.4),
    clusterer=DBSCAN(eps=0.5, min_samples=3)
)

print("✅ Grafo construido")

# Visualizar
fig = plt.figure(figsize=(16, 6))

# 1. Trayectoria en 2D (PCA)
ax1 = fig.add_subplot(131)
scatter = ax1.scatter(brain_proj[:, 0], brain_proj[:, 1], 
                     c=phase_labels, cmap='Set1', s=30, alpha=0.6)
ax1.plot(brain_proj[:, 0], brain_proj[:, 1], 'k-', alpha=0.2, linewidth=0.5)
ax1.set_xlabel('PC1', fontsize=11)
ax1.set_ylabel('PC2', fontsize=11)
ax1.set_title('Trayectoria Cerebral (PCA 2D)', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax1, label='Estado', ticks=[0, 1, 2],
            format=plt.FuncFormatter(lambda x, p: ['Descanso','Atención','Memoria'][int(x)]))

# 2. Trayectoria en 3D (primeras 3 dimensiones originales)
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(brain_traj[:, 0], brain_traj[:, 1], brain_traj[:, 2],
           c=phase_labels, cmap='Set1', s=20, alpha=0.6)
ax2.plot(brain_traj[:, 0], brain_traj[:, 1], brain_traj[:, 2], 
         'k-', alpha=0.2, linewidth=0.5)
ax2.set_title('Trayectoria en Espacio Original (3D)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Neurona 1')
ax2.set_ylabel('Neurona 2')
ax2.set_zlabel('Neurona 3')

# 3. Grafo de Mapper
ax3 = fig.add_subplot(133)

# Convertir a NetworkX para visualizar
G_km = nx.Graph()
for node_id, node_data in mapper_graph_km['nodes'].items():
    G_km.add_node(node_id, size=len(node_data))

for link in mapper_graph_km['links']:
    G_km.add_edge(link[0], link[1])

pos_km = nx.spring_layout(G_km, seed=42, k=1.0)
node_sizes_km = [G_km.nodes[n]['size'] * 10 for n in G_km.nodes()]

# Colorear por fase dominante en cada nodo
node_colors_km = []
for nid in G_km.nodes():
    indices = mapper_graph_km['nodes'][nid]
    dominant_phase = np.median(phase_labels[indices])
    node_colors_km.append(dominant_phase)

nx.draw_networkx_nodes(G_km, pos_km, node_size=node_sizes_km,
                      node_color=node_colors_km, cmap='Set1',
                      vmin=0, vmax=2, alpha=0.8, ax=ax3)
nx.draw_networkx_edges(G_km, pos_km, alpha=0.3, width=2, ax=ax3)

ax3.set_title(f'Grafo de Mapper\n({G_km.number_of_nodes()} nodos, {G_km.number_of_edges()} aristas)',
             fontsize=12, fontweight='bold')
ax3.axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Interpretación:")
print("   El grafo de Mapper revela la estructura CÍCLICA de los estados cerebrales")
print("   Los nodos representan regiones del espacio de estados")
print("   El bucle indica el ciclo: Descanso → Atención → Memoria → Descanso")

---

## 6. Análisis de Bifurcaciones Neuronales

In [ ]:
# Generar trayectoria con bifurcación
print("🧠 Generando trayectoria con BIFURCACIÓN (toma de decisión)...\n")
bifurc_traj, bifurc_labels = generate_brain_trajectory(
    n_timepoints=500,
    n_neurons=30,
    trajectory_type='branching'
)

# Aplicar Mapper
pca_bifurc = PCA(n_components=2)
bifurc_proj = pca_bifurc.fit_transform(bifurc_traj)

mapper_bifurc = mapper.map(
    bifurc_proj,
    bifurc_traj,
    cover=km.Cover(n_cubes=20, perc_overlap=0.3),
    clusterer=DBSCAN(eps=0.3, min_samples=3)
)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Trayectoria
ax1 = axes[0]
scatter = ax1.scatter(bifurc_proj[:, 0], bifurc_proj[:, 1],
                     c=bifurc_labels, cmap='Set2', s=30, alpha=0.6)
ax1.plot(bifurc_proj[:, 0], bifurc_proj[:, 1], 'k-', alpha=0.2, linewidth=0.5)
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('Trayectoria con Bifurcación', fontsize=12, fontweight='bold')
plt.colorbar(scatter, ax=ax1, label='Fase')

# Grafo de Mapper
ax2 = axes[1]
G_bifurc = nx.Graph()
for nid, ndata in mapper_bifurc['nodes'].items():
    G_bifurc.add_node(nid, size=len(ndata))
for link in mapper_bifurc['links']:
    G_bifurc.add_edge(link[0], link[1])

pos_bifurc = nx.spring_layout(G_bifurc, seed=42, k=1.5)
node_sizes_bifurc = [G_bifurc.nodes[n]['size'] * 10 for n in G_bifurc.nodes()]

node_colors_bifurc = []
for nid in G_bifurc.nodes():
    indices = mapper_bifurc['nodes'][nid]
    dominant = np.median(bifurc_labels[indices])
    node_colors_bifurc.append(dominant)

nx.draw_networkx_nodes(G_bifurc, pos_bifurc, node_size=node_sizes_bifurc,
                      node_color=node_colors_bifurc, cmap='Set2',
                      vmin=0, vmax=2, alpha=0.8, ax=ax2)
nx.draw_networkx_edges(G_bifurc, pos_bifurc, alpha=0.3, width=2, ax=ax2)

ax2.set_title(f'Grafo de Mapper: Bifurcación\n(Punto de decisión → 2 ramas)',
             fontsize=12, fontweight='bold')
ax2.axis('off')

plt.tight_layout()
plt.show()

print("\n💡 El grafo revela la BIFURCACIÓN:")
print("   Estado común (verde) → punto de decisión → dos ramas (naranja, morado)")
print("   Esto podría representar: estímulo → proceso cognitivo → decisión A o B")

---

## 7. Ejercicios

### Ejercicio 1: Explora diferentes filtros

Prueba diferentes funciones de filtro y observa cómo cambia el grafo.

---

In [ ]:
# Espacio para Ejercicio 1
# Sugerencias de filtros:
# - Densidad (distancia promedio a vecinos)
# - Distancia a centroide
# - t-SNE
# - Función personalizada


### Ejercicio 2: Mapper en tus datos

Aplica Mapper a datos neuronales reales o simulados más complejos.

---

In [ ]:
# Espacio para Ejercicio 2


## 8. Resumen

### ✅ Aprendimos:

1. **Algoritmo Mapper:** Filtro → Cover → Clustering → Nerve
2. **Implementación:** Desde cero y con KeplerMapper
3. **Aplicaciones:** Trayectorias, bifurcaciones, ciclos
4. **Interpretación:** Nodos = regiones, Aristas = transiciones

### 🧠 Aplicaciones Neurales:

- **Trayectorias de estados** cerebrales
- **Detección de ciclos** cognitivos
- **Puntos de bifurcación** (decisiones)
- **Visualización** de espacios de alta dimensión

---

## Referencias

1. Singh et al. (2007). "Topological methods for the analysis of high dimensional data"
2. Lum et al. (2013). "Extracting insights from the shape of complex data using topology"
3. Saggar et al. (2018). "Towards a new approach to reveal dynamical organization of the brain using TDA"

---

**Próximo:** Tutorial 5 - Series Temporales (EEG/fMRI)

**Autor:** MARK-126  
**Licencia:** MIT